# モデルを評価するための責任あるAIダッシュボードの作成

機械学習モデルを比較・評価する場合、パフォーマンス指標だけでなく、より多くのことを確認したいと思うでしょう。Azure Machine Learningでは、責任あるAIダッシュボードを作成して、データの異なるコホートに対するモデルのパフォーマンスを調べることができます。

## データの準備
責任あるAIダッシュボードを作成するには、Parquetファイルとして保存され、データ資産として登録されたトレーニングデータセットとテストデータセットが必要です。

データは現在CSVファイルとして保存されています。それらをParquetファイルに変換しましょう。


In [1]:
import pandas as pd

# read training and test dataset
df_training = pd.read_csv("train-data/diabetes.csv")
df_test = pd.read_csv("test-data/diabetes-test.csv")

# display the first few rows of the training dataset
df_training.head()

,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,0,171,80,34,23,43.509726,1.213191,21,0
1,8,92,93,47,36,21.240576,0.158365,23,0
2,7,115,47,52,35,41.511523,0.079019,23,0
3,9,103,78,25,304,29.582192,1.282870,43,1
4,1,85,59,27,35,42.604536,0.549542,22,0


In [2]:
import pyarrow as pa
import pyarrow.parquet as pq

# convert data to table
table_training = pa.Table.from_pandas(df_training)
table_test = pa.Table.from_pandas(df_test)

# write tables out to parquet
pq.write_table(table_training, "train-data/diabetes-training.parquet", version="1.0")
pq.write_table(table_test, "test-data/diabetes-test.parquet", version="1.0")

## 続ける前に
このノートブックのコードを実行するには、**azureml-ai-ml**パッケージの最新版が必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [3]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## ワークスペースに接続する
必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [4]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [5]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## データ資産の作成
責任あるAIダッシュボードを作成するには、トレーニングデータセットとテストデータセットを**MLtable**データ資産として登録する必要があります。MLtableデータ資産は、先ほど作成したParquetファイルを参照します。

In [6]:
train_data_path = "train-data/"
test_data_path = "test-data/"
data_version = "1"

In [7]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "diabetes_train_mltable"
input_test_data = "diabetes_test_mltable"

try:
    # Try getting data already registered in workspace
    train_data = ml_client.data.get(
        name=input_train_data,
        version=data_version,
    )
    test_data = ml_client.data.get(
        name=input_test_data,
        version=data_version,
    )
except Exception as e:
    train_data = Data(
        path=train_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI diabetes training data",
        name=input_train_data,
        version=data_version,
    )
    ml_client.data.create_or_update(train_data)

    test_data = Data(
        path=test_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI diabetes test data",
        name=input_test_data,
        version=data_version,
    )
    ml_client.data.create_or_update(test_data)

Uploading train-data (0.7 MBs): 100%|██████████| 698279/698279 [00:00<00:00, 15957340.96it/s]


Uploading test-data (0.35 MBs): 100%|██████████| 353077/353077 [00:00<00:00, 10103995.26it/s]




## AIダッシュボードを作成するためのパイプラインを構築する
ダッシュボードを作成するには、Azure Machine Learningワークスペースにデフォルトで登録されているビルド済みコンポーネントを使用してパイプラインをビルドします。

In [8]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)
print(ml_client_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fe7ace6cf70>,
         subscription_id=6c6683e9-e5fe-4038-8519-ce6ebec2ba15,
         resource_group_name=registry-builtin-prod-eastus-01,
         workspace_name=None)


### モデルの登録
機械学習モデルはすでに学習済みです。このモデルは患者が糖尿病であるかどうかを予測します。すべてのモデルファイルは `model` フォルダに格納されています。

model` フォルダとその中身を指定してモデルを登録する。

In [9]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

Uploading model (0.0 MBs): 100%|██████████| 2296/2296 [00:00<00:00, 80567.92it/s]




## パイプラインの構築
AIダッシュボードを作成するには、あらかじめ用意されているコンポーネントを使用してパイプラインを作成します。どのコンポーネントを使用するか、どの機能をダッシュボードに含めるかを選択できます。エラー分析とモデルの解釈可能性を含むダッシュボードを作成します。

ダッシュボードを構築するためのパイプラインには、担当するAI機能の他に、ダッシュボードを構築するためのコンポーネントと、生成されたすべてのインサイトを収集するためのコンポーネントが必要です。

In [10]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [11]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)#RAI ダッシュボードの構築プロセスを開始するためのコンポーネントです。エラー解析、説明性、カウンターファクチュアル（反事実分析）などをまとめてセットアップします。

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)#エラー解析を行うためのコンポーネントです。

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)#説明性解析を行うためのコンポーネントです。モデルの予測に対する説明（SHAP値など）を生成

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

The current version of RAI built-in components is: 0.17.0


使用したいコンポーネントをすべて取得したら、パイプラインを構築し、適切な順序でコンポーネントを接続します：
1.  ダッシュボードを構築する。  
 エラー分析を追加する  
2. 説明を追加する。    
3. すべての洞察を収集し、ダッシュボードで可視化する。  

In [12]:
# 必要なAzure MLモジュールのインポート
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

# Azure ML上のコンピュートクラスター名
compute_name = "aml-cluster"

# パイプラインの定義（デコレーターを使用）
@dsl.pipeline(
    compute=compute_name,  # パイプラインを実行するコンピュートクラスター
    description="RAI insights on diabetes data",  # パイプラインの説明
    experiment_name=f"RAI_insights_{model_name}",  # 実験名（動的に生成）
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data  # 入力引数として列名、訓練データ、テストデータを受け取る
):
    # Step 1: RAIInsightsを初期化するジョブ
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",  # ダッシュボードのタイトル
        task_type="classification",  # タスクの種類（分類タスク）
        model_info=expected_model_id,  # モデルの情報（事前に定義されたID）
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),  # モデルへの入力パス
        train_dataset=train_data,  # 訓練データセット
        test_dataset=test_data,  # テストデータセット
        target_column_name=target_column_name,  # ターゲット列の名前
    )
    create_rai_job.set_limits(timeout=300)  # ジョブのタイムアウトを300秒に設定

    # Step 2: エラー解析の追加
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,  # RAIダッシュボードを入力として使用
    )
    error_job.set_limits(timeout=300)  # ジョブのタイムアウトを300秒に設定

    # Step 3: 説明性解析の追加
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,  # RAIダッシュボードを入力として使用
        comment="add explanation",  # 説明を追加するコメント
    )
    explanation_job.set_limits(timeout=300)  # ジョブのタイムアウトを300秒に設定

    # Step 4: すべてのインサイトを統合する
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,  # 初期化したRAIダッシュボード
        insight_3=error_job.outputs.error_analysis,  # エラー解析の結果
        insight_4=explanation_job.outputs.explanation,  # 説明性解析の結果
    )
    rai_gather_job.set_limits(timeout=300)  # ジョブのタイムアウトを300秒に設定

    # ダッシュボードの出力をアップロードモードに設定
    rai_gather_job.outputs.dashboard.mode = "upload"

    # 最終的なダッシュボードをパイプラインの出力として返す
    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }


パイプラインが構築されたので、必要な2つの入力、つまりトレーニングとテストのデータセットを定義する必要がある。

In [13]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

最後に、すべてをまとめます：入力をパイプラインに割り当て、ターゲット列（予測ラベル）を設定します。

In [14]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## パイプラインの実行

パイプラインの構築に成功したら、パイプラインを送信できます。以下のコードはパイプラインを送信し、パイプラインのステータスをチェックします。スタジオでパイプラインのステータスを確認することもできます。

In [15]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
pathOnCompute is not a known attribute

Pipeline job can be accessed in the following URL:


When the pipeline is completed, you can review the dashboard in the studio. 